In [1]:
# Importing Basic Libraries:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### X Order Report File:

In [2]:
# Importing the dataset
X_order = pd.read_excel(r'D:\Data\Company X - Order Report.xlsx')
X_order.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


### SKU Master File:

In [3]:
# Loading the weight list:
X_wgt = pd.read_excel(r"D:\Data\Company X - SKU Master.xlsx")

# Checking for the Duplicate Rows:
X_wgt['SKU'].duplicated().sum()

# Dropping the duplicate rows from the table:
X_wgt.drop_duplicates(inplace = True)

# Joining SKU column with X_order Report
lst = []
for i in X_order['SKU']:
    weight = X_wgt['Weight (g)'][X_wgt['SKU'] == i]
    lst.append(int(weight))

X_order['Weight(g)'] = lst

Z = X_order.groupby('ExternOrderNo').sum()

# Changing the weight from grams to kg
Z['Weight(g)'] = round(Z['Weight(g)']/1000,2)

# Renamming columns
Z.rename(columns = {'Weight(g)': 'Weight(kg)'} , inplace = True)

# Rounding off the values to slab values and appending it in Z dataframe:
lst = []
for i in Z['Weight(kg)']:
    num = (round(i - int(i),2))
    if(num < .5 and num != 0):
        diff = (.5 - num)
        s = i + diff
        lst.append(s)
    elif(num > .5):
        diff = 1 - num
        s = i + diff
        lst.append(s)
    elif(num == .5 or num == 0):
        lst.append(i)
        
Z['weight_slab(X)'] =lst

Z.rename(columns = {'Weight(kg)': 'Weight(kg)_x'} , inplace = True)

Z.drop("Order Qty" , axis = 1 , inplace = True)

Z.head()

,Weight(kg)_x,weight_slab(X)
ExternOrderNo,,
2001806210,0.22,0.5
2001806226,0.24,0.5
2001806229,0.50,0.5
2001806232,0.38,0.5
2001806233,0.24,0.5


### Working with the Pincode File:

In [4]:
# Importing pincode file
pin_file = pd.read_excel(r"D:\Data\Company X - Pincode Zones.xlsx")

# Dropping the Warehouse Pincode 
pin_file.drop("Warehouse Pincode" , axis = 1 , inplace = True)

# Renaming the columns
pin_file.rename(columns ={"Customer Pincode" : "Pincode_X"}, inplace = True)

# Setting the index to pincode
pin_file.set_index("Pincode_X" , inplace = True)

pin_file.head()

,Zone
Pincode_X,
507101,d
486886,d
532484,d
143001,b
515591,d


### Working with the Courier Company Report:

In [5]:
# Importing the Courier Invoice Data
df = pd.read_excel(r'D:\Data\Courier Company - Invoice.xlsx')

# Dropping the Warehouse Pincode Column
df.drop('Warehouse Pincode', axis = 1 , inplace = True)

# Rounding off the weights to weight slab
lst = []
for i in df['Charged Weight']:
    num = (round(i - int(i),2))
    if(num < .5 and num != 0):
        diff = (.5 - num)
        s = i + diff
        lst.append(s)
    elif(num > .5):
        diff = 1 - num
        s = i + diff
        lst.append(s)
    elif(num == .5 or num == 0):
        lst.append(i)
        
# Inserting the weight slab column to the lst
df.insert(3 ,"Weight_Slab_C", lst )

# Renaming the columns 
df.rename(columns = {'Zone' : 'Zone(C)',
                     'Charged Weight': "Chareged Weight(C)", 
                     "Billing Amount (Rs.)" : 'Bill_Amount(C)'} , inplace = True)

# Changing the index to the "Order ID"
df.set_index('Order ID',  inplace = True)

df.head()

,AWB Code,Chareged Weight(C),Weight_Slab_C,Customer Pincode,Zone(C),Type of Shipment,Bill_Amount(C)
Order ID,,,,,,,
2001806232,1091117222124,1.30,1.5,507101,d,Forward charges,135.0
2001806273,1091117222194,1.00,1.0,486886,d,Forward charges,90.2
2001806408,1091117222931,2.50,2.5,532484,d,Forward charges,224.6
2001806458,1091117223244,1.00,1.0,143001,b,Forward charges,61.3
2001807012,1091117229345,0.15,0.5,515591,d,Forward charges,45.4


### Concatinating Courier Invoice and X Company's Report:

In [6]:
# Concatinating Courier Invoice and X Company's Report:
df1  = pd.concat([df, Z], axis=1)

# Reseting the index then again assiging index to Customer Pincode:
df1.reset_index(inplace = True)
df1.set_index("Customer Pincode",inplace = True)

# Concatinating the Pincode file with the df1
final_df = pd.concat([df1,pin_file],axis = 1)
final_df.reset_index(inplace = True)

# Calculating the Charges as per X company
lst = []
for index,value in enumerate(final_df['Type of Shipment']):
    if (value == 'Forward charges'):
        if (final_df['Zone'][index] == 'a'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 29.5 + (num -1)*23.6
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)
        
        elif (final_df['Zone'][index] == 'b'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 33 + (num -1)*28.3
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)            
        
        elif (final_df['Zone'][index] == 'c'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 40.1 + (num -1)*38.9
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)            
        
        elif (final_df['Zone'][index] == 'd'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 45.4 + (num -1)*44.8
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)  
            
        elif (final_df['Zone'][index] == 'e'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 58.6 + (num -1)*55.5
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)
            
    elif (value == 'Forward and RTO charges'):
        if (final_df['Zone'][index] == 'a'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 29.5 + 13.6 + 2*(num -1)*23.6 
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)
        
        elif (final_df['Zone'][index] == 'b'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 33 + 20.5 + 2*(num -1)*28.3 
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)            
        
        elif (final_df['Zone'][index] == 'c'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 40.1 + 31.9 + 2*(num -1)*38.9 
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)            
        
        elif (final_df['Zone'][index] == 'd'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 45.4 + 41.3 + 2*(num -1)*44.8 
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)  
            
        elif (final_df['Zone'][index] == 'e'):
            num = final_df['weight_slab(X)'][index]/.5
            tot_charge = 58.6 + 50.7 + 2*(num -1)*55.5
            tot_charge = round(tot_charge,2)
            lst.append(tot_charge)                          

### Making the final Table:

In [7]:
Results = pd.DataFrame()

# Making the Table as Expected:
Results['Order ID'] = final_df['index']
Results['AWB Number'] = final_df['AWB Code']
Results['Total weight as per X (KG)'] = final_df['Weight(kg)_x']
Results['Weight slab as per X (KG)'] = final_df['weight_slab(X)']
Results['Total weight as per Courier Company (KG)'] = final_df['Chareged Weight(C)']
Results['Weight slab charged by Courier Company (KG)'] = final_df['Weight_Slab_C']
Results['Delivery Zone as per X'] = final_df['Zone']
Results['Delivery Zone charged by Courier Company'] = final_df['Zone(C)']
Results['Expected Charge as per X (Rs.)'] = lst
Results['Charges Billed by Courier Company (Rs.)'] = final_df['Bill_Amount(C)']

# Capitalising the Letters of the Zone:
Results['Delivery Zone as per X'] = Results['Delivery Zone as per X'].str.upper()
Results['Delivery Zone charged by Courier Company'] = Results['Delivery Zone charged by Courier Company'].str.upper()

# Calculating the difference
Results['Difference Between Expected Charges and Billed Charges (Rs.)'] = Results['Expected Charge as per X (Rs.)'] - Results['Charges Billed by Courier Company (Rs.)']

# Displaying the results
Results.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,0.38,0.5,1.30,1.5,D,D,45.4,135.0,-89.6
1,2001806273,1091117222194,0.62,1.0,1.00,1.0,D,D,90.2,90.2,0.0
2,2001806408,1091117222931,0.93,1.0,2.50,2.5,D,D,90.2,224.6,-134.4
3,2001806458,1091117223244,0.70,1.0,1.00,1.0,B,B,61.3,61.3,0.0
4,2001807012,1091117229345,0.24,0.5,0.15,0.5,D,D,45.4,45.4,0.0


In [8]:
Results.to_excel("Assignment_final_table.xlsx")

#### Sum of the amount where Courier Company has been charged less than expected

In [36]:
sum_undercharged =Results['Difference Between Expected Charges and Billed Charges (Rs.)'][Results['Difference Between Expected Charges and Billed Charges (Rs.)'] >0].sum()
sum_undercharged

348.99999999999994

#### Count where courier company has been  undercharged :

In [34]:
undercharged_count = sum(Results['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0)
undercharged_count

13

#### Sum of the amount where courier company has been charged more than expected:

In [41]:
amount_overcharged =Results['Difference Between Expected Charges and Billed Charges (Rs.)'][Results['Difference Between Expected Charges and Billed Charges (Rs.)'] <0].sum()
amount_overcharged = abs(amount_overcharged)

#### Count where courier company has been overcharged :

In [29]:
overcharged_count = sum(Results['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0)
overcharged_count

95

#### Sum of the amount where courier company has been correctly charged :

In [27]:
correct_charge = Results['Charges Billed by Courier Company (Rs.)'][Results['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0].sum()
correct_charge

1253.3

#### Count where courier company has been correctly charged :

In [28]:
correct_charge_count = sum(Results['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0)
correct_charge_count

16

In [45]:
amount_file = {
    'Total orders where X has been correctly charged' :{'Count': correct_charge_count,
                                                       'Amount': correct_charge},
    'Total orders where X has been overcharged' :{'Count': overcharged_count,
                                                 'Amount': amount_overcharged},
    'Total orders where X has been undercharged' :{'Count': undercharged_count,
                                                 'Amount': sum_undercharged}    
}

In [47]:
amount_file = pd.DataFrame(amount_file)

In [48]:
amount_file

,Total orders where X has been correctly charged,Total orders where X has been overcharged,Total orders where X has been undercharged
Count,16.0,95.0,13.0
Amount,1253.3,5921.1,349.0


In [49]:
amount_file.to_excel("Assignment_final_table2.xlsx")